# CIS(D)

In [1]:
import numpy as np
import pyci

In [2]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.AOint('6-31g*', 'licn.xyz')
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('mo_scf_info.npz')['eps_a']
Ca = np.load('mo_scf_info.npz')['Ca'] 
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints)
del ao_erints

Size of the ERI tensor will be 0.03 Gb

Ground state SCF Energy : -99.77229345 

Nuclear repulsion energy : 27.84883421 

Total electronic energy : -127.62112766 



In [3]:
aoint.get_orb_info(aoint.wfn)

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 45  
Spin Orbitals     : 90  
Alpha Orbitals    : 8  
Beta Orbitals     : 8  
Occupied Orbitals : 8  
Virtual Orbitals  : 37  



(45, 8, 8, 90, 8, 37)

In [4]:
HCIS = pyci.comp_cis_hamiltonian(eps_a, mo_erints, (8, 37, 45), parallelised=True, ncore=10)

All rows were computed successfully! 



In [5]:
ECIS, CCIS = np.linalg.eigh(HCIS)

In [6]:
mo_so_erints = aoint.eri_mo2so(Ca, Ca)
cis_d_class = pyci.CIS_D(ECIS, CCIS, eps_a, mo_so_erints, (8, 37, 45))

In [9]:
e_cis_d = cis_d_class.comp_cis_d(100, ncores=22)

Ei_d_corr 	 Ei_t_corr 	 Ei_corr 	 Ei_cis(d)
-0.0000 	 0.0000 	 0.0000 	 0.0000
-0.0703 	 0.1044 	 0.0341 	 0.2730
-0.0949 	 0.0785 	 -0.0164 	 0.2253
-0.0949 	 0.0785 	 -0.0164 	 0.2253
-0.0752 	 0.1020 	 0.0268 	 0.2747
-0.0752 	 0.1020 	 0.0268 	 0.2747
-0.1076 	 0.0481 	 -0.0595 	 0.2035
-0.1105 	 0.0674 	 -0.0431 	 0.2460
-0.1105 	 0.0674 	 -0.0431 	 0.2460
-0.0972 	 0.0751 	 -0.0220 	 0.2807
-0.0831 	 0.0901 	 0.0070 	 0.3372
-0.0806 	 0.0804 	 -0.0002 	 0.3321
-0.0806 	 0.0804 	 -0.0002 	 0.3321
-0.0828 	 0.0937 	 0.0110 	 0.3464
-0.0828 	 0.0937 	 0.0110 	 0.3464
-0.1030 	 0.0744 	 -0.0286 	 0.3406
-0.1030 	 0.0744 	 -0.0286 	 0.3406
-0.1184 	 0.0524 	 -0.0660 	 0.3165
-0.1827 	 0.0535 	 -0.1292 	 0.2631
-0.1381 	 0.0577 	 -0.0805 	 0.3372
-0.1381 	 0.0577 	 -0.0805 	 0.3372
-0.1034 	 0.0731 	 -0.0303 	 0.3928
-0.0986 	 0.0755 	 -0.0232 	 0.4093
-0.0986 	 0.0755 	 -0.0232 	 0.4093
-0.0950 	 0.0799 	 -0.0151 	 0.4340
-0.0946 	 0.0809 	 -0.0137 	 0.4357
-0.0946 	 0.0809 	 -0.0137 